In [15]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from collections import defaultdict
import matplotlib.pyplot as plt
plt.rcParams.update({
    'axes.titlesize': 20,
    'axes.labelsize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold'
})

import json
import csv 
import math 

In [16]:
# --- CONFIGURATION ---
MIN_STOP_TIME = 0.5
MIN_HEADWAY = 3.0
BIG_M = 2000
TOLERANCE_PENALTY_WEIGHT = 600.0

# --- SOLVER PARAMETERS ---
NEW_TIME_LIMIT = 600
NEW_MIP_GAP = 0.001  # 0.1% relative gap
FEASIBILITY_TOL = 1e-5  # 0.00001 minutes absolute tolerance

# --- STATION AND COLUMN DEFINITIONS ---
STATIONS_ORDER = ['CCG', 'MCT', 'DDR', 'BA', 'ADH', 'GMN', 'BVI', 'BYR', 'BSR', 'VR', 'DRD']
OUTPUT_COLS_STATION_EVENTS = [
    'CCGa', 'CCGd', 'MCTa', 'MCTd', 'DDRa', 'DDRd', 'BAa', 'BAd', 'ADHa', 'ADHd',
    'GMNa', 'GMNd', 'BVIa', 'BVId', 'BYRa', 'BYRd', 'BSRa', 'BSRd', 'VRa', 'VRd',
    'DRDa', 'DRDd']

In [17]:
def load_and_validate_data():
    """Load all data files and validate consistency"""
    print("Loading data files...")
    
    # Load CSC file
    csc_df = pd.read_csv('1-o-event-ids.csv')
    print(f"Loaded CSC file: {len(csc_df)} services")
    
    # Load patterns
    patterns_df = pd.read_csv('OWR-patterns-consolidated-sequential.csv')
    print(f"Loaded patterns: {len(patterns_df)} patterns")
    
    # Load constraints
    with open('constraints_f_s.json', 'r') as f:
        constraints = json.load(f)
    print("Loaded constraints file")
    
    return csc_df, patterns_df, constraints

In [20]:
def extract_travel_times(patterns_df):
    """Extract travel times between stations from patterns - create bidirectional mapping"""
    travel_times = {}
    
    for _, row in patterns_df.iterrows():
        pattern_id = row['Pattern_ID']
        travel_times[pattern_id] = {}
        
        # Process all segments
        for i in range(1, 11):
            seg_col = f'Major_Segment_{i}'
            time_col = f'Time_{i}'
            
            if pd.notna(row[seg_col]) and pd.notna(row[time_col]):
                segment = str(row[seg_col]).strip()
                if '-' in segment:
                    from_st, to_st = segment.split('-')
                    travel_time = float(row[time_col])
                    
                    # Store both directions with same travel time
                    travel_times[pattern_id][f"{from_st}-{to_st}"] = travel_time
                    travel_times[pattern_id][f"{to_st}-{from_st}"] = travel_time  # Reverse direction
                    
                    print(f"  {pattern_id}: {from_st}->{to_st} = {travel_time} min (bidirectional)")
    
    return travel_times
print(extract_travel_times(patterns_df))


NameError: name 'patterns_df' is not defined